In [1]:

from llama_index.core.retrievers import NLSQLRetriever

import sqlite3
import os
import llama_index
import sqlalchemy
import openai

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI


from sqlalchemy import text


In [2]:
import os

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key


engine = create_engine("sqlite:///classes.db")

llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
sql_database = SQLDatabase(engine, include_tables=["class_data"])


In [4]:
#Establishing engines and connections

conn = sqlite3.connect('classes.db')
cursor = conn.cursor()


engine = create_engine("sqlite:///classes.db")

llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
sql_database = SQLDatabase(engine, include_tables=["class_data"])


In [5]:
#sqlite query
cursor.execute("PRAGMA table_info(class_data)")
tables = cursor.fetchall()
for col in tables:
    print(col[1], col[2])

course_campus TEXT
course_credits TEXT
course_description TEXT
course_id TEXT
course_offered TEXT
course_title TEXT
department_abbrev TEXT
is_bottleneck TEXT
is_gateway TEXT
prereq_string TEXT
course_coi REAL
course_level_coi REAL
curric_coi REAL
percent_in_range REAL
autumn INTEGER
winter INTEGER
spring INTEGER
summer INTEGER
0 REAL
1 REAL
2 REAL
3 REAL
4 REAL
5 REAL
6 REAL
7 REAL
8 REAL
9 REAL
10 REAL
11 REAL
12 REAL
13 REAL
14 REAL
15 REAL
16 REAL
17 REAL
18 REAL
19 REAL
20 REAL
21 REAL
22 REAL
23 REAL
24 REAL
25 REAL
26 REAL
27 REAL
28 REAL
29 REAL
30 REAL
31 REAL
32 REAL
33 REAL
34 REAL
35 REAL
36 REAL
37 REAL
38 REAL
39 REAL
40 REAL
mean REAL
median REAL
sd REAL
fourpointo_per REAL


In [31]:

#sqlalchemy query

with engine.connect() as con:
    rows = con.execute(text("SELECT COUNT(course_id) \nFROM class_data\nWHERE course_credits like '%4.0%' AND department_abbrev LIKE '%CS%' AND mean > 3.0"))
    for row in rows:
        print(row)

(39,)


In [7]:
#llama index retriever

nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["class_data"], return_raw=True
)


In [35]:
#llama index query retriever


query = """Given this context about our SQL schema, create a viable sql lite query from the question. 
Context: ALWAYS use LIKE for all WHERE statements. ALWAYS wrap WHERE conditions with the '%' operator on both sides no matter what.
course_credits column is of dtype
text in float format. 
Department_abbrev column is of dtype text and represents the type of class it is ie. CS, ANTH, BIO.
Question:
How many 4 credit Computer science classes are there with a GPA mean greater than 3.0?"""
 

response = nl_sql_retriever.retrieve(query)
print(response)


[NodeWithScore(node=TextNode(id_='043f35f4-e587-4fd3-b932-99384ba8a9a3', embedding=None, metadata={'sql_query': "SELECT COUNT(*) \nFROM class_data\nWHERE course_credits LIKE '4%' \nAND department_abbrev LIKE 'CS' \nAND mean > 3.0", 'result': [(0,)], 'col_keys': ['COUNT(*)']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='[(0,)]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)]
